In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn

In [3]:
import easydict 
args = easydict.EasyDict({ "batch_size": 6, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':2,
                         'output_dir' :'../trained_model/fasterrcnn_resnet50_fpn/'})

In [4]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [5]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [7]:
num_classes = 2
model = fasterrcnn_resnet50_fpn(pretrained=True, min_size=1024, max_size=1024)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda')
model.to(device)
# print('model is loaded to gpu')

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(1024,), max_size=1024, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU

In [7]:
# # pretrained.state_dict()
# for p in model.parameters() :
#     if p.requires_grad == True :
#         print(p.shape)

In [8]:
# model.head.regression_head

In [9]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [10]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(120):
    train_one_epoch(model, optimizer, train_loader, device, epoch, 600)
    lr_scheduler.step()
    
    if epoch > 20 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/503]  eta: 0:14:26  lr: 0.001000  loss: 1.7098 (1.7098)  loss_classifier: 1.4015 (1.4015)  loss_box_reg: 0.0050 (0.0050)  loss_objectness: 0.2905 (0.2905)  loss_rpn_box_reg: 0.0128 (0.0128)  time: 1.7234  data: 1.0487  max mem: 7081
Epoch: [0]  [502/503]  eta: 0:00:00  lr: 0.001000  loss: 0.1116 (0.1587)  loss_classifier: 0.0249 (0.0326)  loss_box_reg: 0.0058 (0.0099)  loss_objectness: 0.0563 (0.0897)  loss_rpn_box_reg: 0.0054 (0.0266)  time: 0.6875  data: 0.0143  max mem: 7348
Epoch: [0] Total time: 0:05:51 (0.6995 s / it)
Epoch: [1]  [  0/503]  eta: 0:14:27  lr: 0.001000  loss: 0.1228 (0.1228)  loss_classifier: 0.0358 (0.0358)  loss_box_reg: 0.0036 (0.0036)  loss_objectness: 0.0748 (0.0748)  loss_rpn_box_reg: 0.0086 (0.0086)  time: 1.7240  data: 1.0348  max mem: 7348
Epoch: [1]  [502/503]  eta: 0:00:00  lr: 0.001000  loss: 0.1006 (0.1178)  loss_classifier: 0.0307 (0.0315)  loss_box_reg: 0.0105 (0.0110)  loss_objectness: 0.0531 (0.0639)  loss_rpn_box_reg: 0.0041 (0.01

In [11]:
evaluate(model, test_loader, device=device) 

creating index...
index created!
Test:  [  0/168]  eta: 0:03:48  model_time: 0.3092 (0.3092)  evaluator_time: 0.0089 (0.0089)  time: 1.3622  data: 1.0147  max mem: 7348
Test:  [100/168]  eta: 0:00:29  model_time: 0.2859 (0.2847)  evaluator_time: 0.0065 (0.0069)  time: 0.4329  data: 0.1307  max mem: 7348
Test:  [167/168]  eta: 0:00:00  model_time: 0.2879 (0.2851)  evaluator_time: 0.0068 (0.0070)  time: 0.4143  data: 0.1165  max mem: 7348
Test: Total time: 0:01:12 (0.4325 s / it)
Averaged stats: model_time: 0.2879 (0.2851)  evaluator_time: 0.0068 (0.0070)
Accumulating evaluation results...
DONE (t=0.18s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.196
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.059
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=med